In [ ]:
# Imports
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import tkinter as tk
from tkinter import ttk
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os

# Feature Engineering: Technical Indicators
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df

# Data Collection: 6 Years of Data with CSV Saving
def get_crypto_data(symbol, years=6):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years * 365)
    crypto = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), 
                        end=end_date.strftime('%Y-%m-%d'), auto_adjust=False)
    crypto = calculate_technical_indicators(crypto)
    # Save to CSV
    csv_filename = f"{symbol}_data.csv"
    crypto.to_csv(csv_filename)
    print(f"Saved data to {csv_filename}")
    return crypto.dropna()

# Data Preparation with Multiple Features
def prepare_data(df, look_back=60, features=['Close', 'SMA_20', 'SMA_50', 'EMA_12', 'RSI', 'MACD', 'MACD_Signal']):
    data = df[features]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    
    X, y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i-look_back:i])
        y.append(scaled_data[i, 0])  # Predict 'Close' price
    
    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[0:train_size], X[train_size:]
    y_train, y_test = y[0:train_size], y[train_size:]
    
    return X_train, y_train, X_test, y_test, scaler

# LSTM Model with Input Layer
def create_lstm_model(look_back, feature_count):
    model = Sequential([
        Input(shape=(look_back, feature_count)),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(32),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Future Prediction
def predict_future(model, last_sequence, scaler, days_ahead, feature_count):
    future_predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(days_ahead):
        next_pred = model.predict(current_sequence.reshape(1, *current_sequence.shape), verbose=0)
        future_predictions.append(next_pred[0, 0])
        current_sequence = np.roll(current_sequence, -1, axis=0)
        new_row = np.zeros((feature_count,))
        new_row[0] = next_pred[0, 0]
        current_sequence[-1] = new_row
    
    future_array = np.zeros((len(future_predictions), feature_count))
    future_array[:, 0] = future_predictions
    return scaler.inverse_transform(future_array)[:, 0]

# GUI Application
class CryptoPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Crypto Price Predictor")
        self.root.geometry("1000x700")
        
        # Crypto selection
        tk.Label(root, text="Select Cryptocurrency:", font=('Arial', 12)).pack(pady=5)
        self.crypto_var = tk.StringVar()
        crypto_options = {"Bitcoin": "BTC-USD", "Ethereum": "ETH-USD", "Ripple": "XRP-USD"}
        self.crypto_dropdown = ttk.Combobox(root, textvariable=self.crypto_var, 
                                          values=list(crypto_options.keys()), font=('Arial', 10))
        self.crypto_dropdown.pack()
        self.crypto_symbols = crypto_options
        
        # Date entry
        tk.Label(root, text="Prediction Date (YYYY-MM-DD):", font=('Arial', 12)).pack(pady=5)
        self.date_entry = tk.Entry(root, font=('Arial', 10))
        self.date_entry.pack()
        
        # Predict button
        tk.Button(root, text="Predict Price", command=self.make_prediction, 
                 font=('Arial', 12), bg='lightblue').pack(pady=10)
        
        # Result display
        self.result_label = tk.Label(root, text="", font=('Arial', 14, 'bold'))
        self.result_label.pack(pady=10)
        
        # Plot canvas
        self.fig, self.ax = plt.subplots(figsize=(10, 5))
        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.get_tk_widget().pack()

    def make_prediction(self):
        try:
            crypto = self.crypto_symbols[self.crypto_var.get()]
            target_date = datetime.strptime(self.date_entry.get(), '%Y-%m-%d')
            current_date = datetime.now()
            days_ahead = (target_date - current_date).days
            
            if days_ahead <= 0:
                self.result_label.config(text="Please select a future date!")
                return
                
            data = get_crypto_data(crypto, years=6)
            features = ['Close', 'SMA_20', 'SMA_50', 'EMA_12', 'RSI', 'MACD', 'MACD_Signal']
            look_back = 60
            
            X_train, y_train, X_test, y_test, scaler = prepare_data(data, look_back, features)
            model = create_lstm_model(look_back, len(features))
            model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
            
            last_sequence = scaler.transform(data[features][-look_back:])
            predictions = predict_future(model, last_sequence, scaler, days_ahead, len(features))
            
            predicted_price = predictions[-1]
            self.result_label.config(text=f"Predicted {self.crypto_var.get()} price on "
                                   f"{target_date.strftime('%Y-%m-%d')}: ${predicted_price:,.2f}")
            
            self.ax.clear()
            dates = pd.date_range(end=target_date, periods=days_ahead)
            self.ax.plot(data.index[-100:], data['Close'][-100:], label='Historical', color='blue')
            self.ax.plot(dates, predictions, label='Predicted', color='orange', linestyle='--')
            self.ax.set_title(f"{self.crypto_var.get()} Price Prediction", fontsize=14)
            self.ax.set_xlabel("Date", fontsize=12)
            self.ax.set_ylabel("Price (USD)", fontsize=12)
            self.ax.legend()
            self.ax.grid(True)
            self.canvas.draw()
            
        except Exception as e:
            self.result_label.config(text=f"Error: {str(e)}")

# Main Execution
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = CryptoPredictorApp(root)
        root.mainloop()
    except ImportError as e:
        print(f"Missing dependency: {str(e)}")
        print("Please install: pip install numpy pandas yfinance tensorflow scikit-learn matplotlib")

[*********************100%***********************]  1 of 1 completed


Saved data to XRP-USD_data.csv


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tkinter as tk
from tkinter import ttk
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os

In [5]:
# Data Collection: 6 Years of Data with CSV Saving
def get_crypto_data(symbol, years=6):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years * 365)
    crypto = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), 
                        end=end_date.strftime('%Y-%m-%d'))
    crypto = calculate_technical_indicators(crypto)
    # Save to CSV
    csv_filename = f"{symbol}_data.csv"
    crypto.to_csv(csv_filename)
    print(f"Saved data to {csv_filename}")
    return crypto.dropna()

In [6]:
# Feature Engineering: Technical Indicators
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df

In [7]:
# Data Preparation with Multiple Features
def prepare_data(df, look_back=60, features=['Close', 'SMA_20', 'SMA_50', 'EMA_12', 'RSI', 'MACD', 'MACD_Signal']):
    data = df[features]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    
    X, y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i-look_back:i])
        y.append(scaled_data[i, 0])  # Predict 'Close' price
    
    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[0:train_size], X[train_size:]
    y_train, y_test = y[0:train_size], y[train_size:]
    
    return X_train, y_train, X_test, y_test, scaler

In [8]:
# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(32),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [9]:
# Future Prediction
def predict_future(model, last_sequence, scaler, days_ahead, feature_count):
    future_predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(days_ahead):
        next_pred = model.predict(current_sequence.reshape(1, *current_sequence.shape), verbose=0)
        future_predictions.append(next_pred[0, 0])
        current_sequence = np.roll(current_sequence, -1, axis=0)
        new_row = np.zeros((feature_count,))
        new_row[0] = next_pred[0, 0]
        current_sequence[-1] = new_row
    
    future_array = np.zeros((len(future_predictions), feature_count))
    future_array[:, 0] = future_predictions
    return scaler.inverse_transform(future_array)[:, 0]

In [10]:
# GUI Application
class CryptoPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Crypto Price Predictor")
        self.root.geometry("1000x700")
        
        # Crypto selection
        tk.Label(root, text="Select Cryptocurrency:", font=('Arial', 12)).pack(pady=5)
        self.crypto_var = tk.StringVar()
        crypto_options = {"Bitcoin": "BTC-USD", "Ethereum": "ETH-USD", "Ripple": "XRP-USD"}
        self.crypto_dropdown = ttk.Combobox(root, textvariable=self.crypto_var, 
                                          values=list(crypto_options.keys()), font=('Arial', 10))
        self.crypto_dropdown.pack()
        self.crypto_symbols = crypto_options
        
        # Date entry
        tk.Label(root, text="Prediction Date (YYYY-MM-DD):", font=('Arial', 12)).pack(pady=5)
        self.date_entry = tk.Entry(root, font=('Arial', 10))
        self.date_entry.pack()
        
        # Predict button
        tk.Button(root, text="Predict Price", command=self.make_prediction, 
                 font=('Arial', 12), bg='lightblue').pack(pady=10)
        
        # Result display
        self.result_label = tk.Label(root, text="", font=('Arial', 14, 'bold'))
        self.result_label.pack(pady=10)
        
        # Plot canvas
        self.fig, self.ax = plt.subplots(figsize=(10, 5))
        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.get_tk_widget().pack()

    def make_prediction(self):
        try:
            crypto = self.crypto_symbols[self.crypto_var.get()]
            target_date = datetime.strptime(self.date_entry.get(), '%Y-%m-%d')
            current_date = datetime.now()
            days_ahead = (target_date - current_date).days
            
            if days_ahead <= 0:
                self.result_label.config(text="Please select a future date!")
                return
                
            data = get_crypto_data(crypto, years=6)
            features = ['Close', 'SMA_20', 'SMA_50', 'EMA_12', 'RSI', 'MACD', 'MACD_Signal']
            look_back = 60
            
            X_train, y_train, X_test, y_test, scaler = prepare_data(data, look_back, features)
            model = create_lstm_model((look_back, len(features)))
            model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
            
            last_sequence = scaler.transform(data[features][-look_back:])
            predictions = predict_future(model, last_sequence, scaler, days_ahead, len(features))
            
            predicted_price = predictions[-1]
            self.result_label.config(text=f"Predicted {self.crypto_var.get()} price on "
                                   f"{target_date.strftime('%Y-%m-%d')}: ${predicted_price:,.2f}")
            
            self.ax.clear()
            dates = pd.date_range(end=target_date, periods=days_ahead)
            self.ax.plot(data.index[-100:], data['Close'][-100:], label='Historical', color='blue')
            self.ax.plot(dates, predictions, label='Predicted', color='orange', linestyle='--')
            self.ax.set_title(f"{self.crypto_var.get()} Price Prediction", fontsize=14)
            self.ax.set_xlabel("Date", fontsize=12)
            self.ax.set_ylabel("Price (USD)", fontsize=12)
            self.ax.legend()
            self.ax.grid(True)
            self.canvas.draw()
            
        except Exception as e:
            self.result_label.config(text=f"Error: {str(e)}")

In [ ]:
# Main Execution
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = CryptoPredictorApp(root)
        root.mainloop()
    except ImportError as e:
        print(f"Missing dependency: {str(e)}")
        print("Please install: pip install numpy pandas yfinance tensorflow scikit-learn matplotlib")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Saved data to ETH-USD_data.csv


C:\Users\ravis\OneDrive\Desktop\SAHIL_PROFILE\IMPORTANT\Cryptovision\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
